<a href="https://colab.research.google.com/github/Sounakray2003/Asmadiya-tech/blob/main/NeMoGuardrails.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
# Reinstall nemoguardrails[all] to ensure dependencies are available
!pip install nemoguardrails

In [3]:
# --------------------------------------------------------------
# ONE-CELL: NeMo Guardrails + Llama 3.1 8B + GRADIO (MODEL RESPONDS!)
# --------------------------------------------------------------

import os, textwrap, subprocess, time, nest_asyncio, requests
from pathlib import Path
from IPython.display import display, clear_output
import gradio as gr
nest_asyncio.apply()

# 1. Install
!apt-get update -qq > /dev/null
!apt-get install -y -qq curl wget git > /dev/null
!pip install -q nemoguardrails==0.9.0 langchain-ollama gradio nest-asyncio > /dev/null

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
ERROR: Cannot install langchain-ollama==0.1.0, nemoguardrails and nemoguardrails==0.9.0 because these package versions have conflicting dependencies.
ERROR: ResolutionImpossible: for help visit https://pip.pypa.io/en/latest/topics/dependency-resolution/#dealing-with-dependency-conflicts


In [ ]:


# 2. Install Ollama
print("Installing Ollama...")
!curl -fsSL https://ollama.com/install.sh | sh > /dev/null 2>&1
clear_output()


In [5]:

# 3. Start Ollama
print("Starting Ollama...")
ollama_process = subprocess.Popen(["ollama", "serve"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

# Wait
print("Waiting for Ollama...")
for _ in range(30):
    try:
        requests.get("http://localhost:11434", timeout=2)
        break
    except:
        time.sleep(2)
else:
    raise Exception("Ollama failed")

Starting Ollama...
Waiting for Ollama...


In [ ]:
# 4. Pull VALID model
print("Pulling Llama 3 8B (q5_K_M)...")
result = subprocess.run(["ollama", "pull", "llama3:8b-instruct-q5_K_M"], capture_output=True, text=True)
if result.returncode != 0:
    print("Pull failed:", result.stderr)
    raise Exception("Model pull failed")
clear_output()
print("Model ready")

In [7]:
# 4. Verify model exists
print("Checking model list...")
models = requests.get("http://localhost:11434/api/tags").json()
model_names = [m["name"] for m in models.get("models", [])]
print("Available models:", model_names)

if "llama3:8b-instruct-q5_K_M" not in model_names:
    raise Exception("Model not found after pull!")

# 5. TEST LLM CALL
print("\nTesting LLM with 'Hello'...")
payload = {
    "model": "llama3:8b-instruct-q5_K_M",
    "prompt": "Hello, who are you?",
    "stream": False
}
response = requests.post("http://localhost:11434/api/generate", json=payload)
if response.status_code != 200:
    print("LLM CALL FAILED:", response.text)
    raise Exception("Ollama API failed")
else:
    result = response.json()
    print("MODEL IS WORKING!")
    print("Response:", result["response"])

# --------------------------------------------------------------
# If you see "MODEL IS WORKING!" + a response → OLLAMA IS FINE
# --------------------------------------------------------------

clear_output()
print("MODEL TEST PASSED!")
print("Response from Llama 3:")
print(result["response"])

MODEL TEST PASSED!
Response from Llama 3:
Nice to meet you! I am LLaMA, an AI assistant developed by Meta AI that can understand and respond to human input in a conversational manner. I'm not a human, but a computer program designed to simulate conversation and answer questions to the best of my knowledge based on my training data.

I can chat with you about a wide range of topics, from science and history to entertainment and culture. I can also help with tasks such as language translation, generating text, and providing definitions for words and phrases.

So, what's on your mind? Is there something specific you'd like to talk about or ask me?


In [ ]:
def ollama_chat(message, history):
    payload = {
        "model": "llama3:8b-instruct-q5_K_M",
        "prompt": message,
        "stream": False
    }
    r = requests.post("http://localhost:11434/api/generate", json=payload)
    return r.json()["response"] if r.ok else f"Error {r.status_code}"

# ------------------------------------------------------------------
# 7. Launch Gradio UI
# ------------------------------------------------------------------
print("\nLaunching Gradio UI …")
gr.ChatInterface(
    fn=ollama_chat,
    title="Llama 3 8B (direct Ollama)",
    description="Chat with the model **without any Guardrails**.",
    examples=["Hello", "Write a Python function", "Explain transformers"],
    submit_btn="Send"
).launch(share=True, debug=False)

In [ ]:
# ------------------------------------------------------------------
# 1. Check what model you have (IMPORTANT!)
# ------------------------------------------------------------------
print("Checking available models...")
try:
    models = requests.get("http://localhost:11434/api/tags").json()
    model_list = [m["name"] for m in models.get("models", [])]
    print("Found models:", model_list)
except:
    model_list = []
    print("No models found or Ollama not running")

# ------------------------------------------------------------------
# 2. Pick the first available model (or set manually)
# ------------------------------------------------------------------
if not model_list:
    raise Exception("No model found! Run: !ollama pull llama3")

# Use the first model (or change to your preferred one)
MODEL_NAME = model_list[0]  # ← Change this if you want a specific one
print(f"Using model: {MODEL_NAME}")

In [ ]:
# 5. Create config + rails.co (EXACTLY as you provided)
proj = Path("guardrails_project/config")
proj.mkdir(parents=True, exist_ok=True)

cfg = """
models:
  - type: main
    engine: ollama
    model: llama3:8b-instruct-q5_K_M
    parameters:
      base_url: http://localhost:11434
      temperature: 0.5
"""
(proj.parent / "config.yml").write_text(textwrap.dedent(cfg).strip())

rails_co = '''
# ========================================
# FINANCE-ONLY BOT – NO GENERAL KNOWLEDGE
# ========================================

# === PII / CREDENTIALS BLOCK ===
define user ask pii
  "credit card" "SSN" "password" "account" "CVV" "PIN"

define bot block pii
  "I cannot discuss sensitive account information."

define flow pii_block
  when user ask pii
    bot block pii
    stop

# === FRAUD / ILLEGAL ===
define user ask fraud
  "fake" "scam" "phish" "mule" "insider" "wire fraud"

define bot block fraud
  "I cannot assist with illegal activities."

define flow fraud_block
  when user ask fraud
    bot block fraud
    stop

# === JAILBREAK ===
define user jailbreak
  "DAN" "ignore" "bypass" "override"

define bot block jailbreak
  "I cannot override safety rules."

define flow jailbreak_block
  when user jailbreak
    bot block jailbreak
    stop

# === NON-FINANCE / GENERAL KNOWLEDGE ===
define user ask non finance
  "mango" "apple" "fruit" "animal" "plant" "weather"
  "joke" "movie" "song" "celebrity" "sport" "game"
  "history" "science" "biology" "physics" "math"
  "what is" "define" "explain" unless contains "finance" "money" "bank" "invest"

define bot redirect finance
  "I'm a financial assistant. I can help with banking, investments, loans, or credit. What would you like to know?"

define flow non_finance_block
  when user ask non finance
    bot redirect finance
    stop

# === FINANCE ADVICE (WITH DISCLAIMER) ===
define user ask finance advice
  "should I" "buy" "sell" "invest" "loan" "mortgage" "bitcoin"

define bot disclaim
  "This is not financial advice. Please consult a licensed professional."

define flow finance_advice
  when user ask finance advice
    bot disclaim
    bot generate

# === PURE FINANCE QUESTIONS (NO DISCLAIMER NEEDED) ===
define user ask pure finance
  "compound interest" "APR" "APY" "401k" "IRA" "credit score"
  "diversification" "ETF" "mutual fund" "bond" "stock"

define flow pure_finance
  when user ask pure finance
    bot generate

# === DEFAULT: CATCH ALL FINANCE-RELATED ===
define flow general
  user $user_message
  bot generate
'''
(proj / "rails.co").write_text(textwrap.dedent(rails_co).strip())
clear_output()
print("Guardrails ready")

In [11]:
# 6. Load
from nemoguardrails import LLMRails, RailsConfig
rails = LLMRails(RailsConfig.from_path("guardrails_project"))
clear_output()
print("Guardrails loaded")

Guardrails loaded


In [ ]:
# 7. Gradio
def chat(message, history):
    try:
        resp = rails.generate(messages=[{"role": "user", "content": message}])
        return resp["content"]
    except Exception as e:
        return f"Error: {e}"

print("Launching Gradio...")
gr.ChatInterface(
    fn=chat,
    title="Llama 3.1 + Guardrails",
    examples=["hello", "Write Python code", "Explain AI"],
    submit_btn="Send"
).launch(share=True)